In [ ]:
import os
import struct
import pandas as pd
import numpy as np
import time, datetime
import sys, getopt
# import multiprocessing
# import talib as tdx
from easyquant import MongoIo
import matplotlib.pyplot as plt
# from easyquant.indicator.base import *
# import QUANTAXIS as QA

import talib
import matplotlib.ticker as ticker
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import mplfinance as mpf
from matplotlib import gridspec


In [ ]:
def getStockByDate(stockData, date,code):
    if date < stockData.index[0][0]:
        return None
    try:
        return stockData.loc[date,code]
    except:
        return getStockByDate(stockData, date - datetime.timedelta(1), code)

def getValidDate(year, month, day, nextday = True):
    try:
        return pd.Timestamp(year, month, day)
    except:
        # print('getValidDate', year, month, day)
        if nextday:
            return getValidDate(year, month + 1, 1)
        else:
            return getValidDate(year, month, day - 1)
          
def beforeDate(calcDate, year = 0, month = 0):
    year = year + int(month / 12)
    month = month - int(month / 12) * 12
    if calcDate.month > month:
        result = getValidDate(calcDate.year - year, calcDate.month - month, calcDate.day)
    else:
        result = getValidDate(calcDate.year - year - 1, calcDate.month + 12 - month, calcDate.day)
    return result
  
def afterDate(calcDate, year = 0, month = 0):
    year = year + int(month / 12)
    month = month - int(month / 12) * 12
    if calcDate.month + month > 12:
        result = getValidDate(calcDate.year + year + 1, calcDate.month + month - 12, calcDate.day)
    else:
        result = getValidDate(calcDate.year + year, calcDate.month + month, calcDate.day)
    return result

def calcBAYMDateLst(calcDate, dstDate, year, month, before = True):
    out = []
    if before:
        value = eval('beforeDate')(calcDate, year, month)
        out.append(value)
        while value > dstDate:
            calcDate = value
            value = eval('beforeDate')(calcDate, year, month)
            out.append(value)
    else:
        value = eval('afterDate')(calcDate, year, month)
        out.append(value)
        while value < dstDate:
            calcDate = value
            value = eval('afterDate')(calcDate, year, month)
            out.append(value)
    return out

def calcBAYMDate(calcDate, year, month, before = True):
    if before:
        value = eval('beforeDate')(calcDate, year, month)
    else:
        value = eval('afterDate')(calcDate, year, month)
    return value


In [ ]:
def GannCheck(dataIn, code, lastDay, posB0Day, delta = 0.1, gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}]):
  #     data=dataIn.reset_index()
#     data=data.set_index(['date'])
#     lstDay = data.index[-1]
    # code = data.index[-1][1] ##二维索引用
    # lastDay = data.index[-1][0] ##数据最大日期
    firstDay = data.index[0][0]
    x0data = getStockByDate(data, posB0Day, code) #基础点1的数据
    xedata = getStockByDate(data, lastDay, code) #基础点1的数据 
    x1wight = 1 ##权重
    pxedata = 0
    pdelta = 0
    tj0=gannList[0] ##基础计算目标
    ##基础点2
    posB1Day = calcBAYMDate(posB0Day, gannBase['y'], gannBase['m'], False)
    if firstDay > posB0Day:
        print('checkLoop1', firstDay, posB0Day, posB1Day)
        return GannCheck(dataIn, code, lastDay, posB1Day, delta, gannBase, gannList)
    pos0Day = None
    pos1Day = None
    if posB1Day > lastDay:
        print('checkLoop2', firstDay, posB0Day, posB1Day)
        result = GannSubCheck(data, code, lastDay, posB1Day, gannList, delta)
        if result != None:
            result['posB0'] = posB0Day
            result['posB1'] = posB1Day
        return result
    else:
        print('checkLoop2', posB1Day)
        result = GannSubCheck(data, code, lastDay, posB0Day, [gannBase])
        if result != None:
            result['posB0'] = posB0Day
            result['posB1'] = posB1Day
        return result

        # pos0Day = posB0Day
        # pos1Day = posB1Day
        # x0data = getStockByDate(data, pos0Day, code)
        # x1data=getStockByDate(data, pos1Day, code)
        # x1wight = x1data.close / x0data.close
        # pxedata = getStockByDate(data, calcBAYMDate(lastDay, gannBase['y'], gannBase['m'], True), code) * x1wight
        # pdelta = abs((pxedata.close - xedata.close ) / xedata.close )
        # return {'posB0':posB0Day, 'posB1':posB1Day, 'pos0':pos0Day, 'pos1':pos1Day, 'w':x1wight, 'pdata':pxedata, 'pd':pdelta}


In [ ]:
def GannSubCheck(data, code, lastDay, posB1Day, gannList, delta = 0.1):
    # code = data.index[-1][1] ##二维索引用
    # lastDay = data.index[-1][0] ##数据最大日期
    xedata = getStockByDate(data, lastDay, code) #基础点1的数据
    firstDay = data.index[0][0]
    tj1=gannList[0]
    # print(tj1)
    print('GannSubCheck0', lastDay, posB1Day, gannList)
    pos0Day = None
    pos1Day = None
    x1wight = 1 ##权重
    pxedata = 0
    pdelta = 0
    base2 = calcBAYMDate(posB1Day, tj1['y'], tj1['m'], True)
    base1 = calcBAYMDate(base2, tj1['y'], tj1['m'], True)
    if base2 < lastDay and base1 < lastDay and base1 > data.index[0][0]:
        pos0Day = base1
        pos1Day = base2
        x0data = getStockByDate(data, pos0Day, code)
        x1data = getStockByDate(data, pos1Day, code)
        # print(pos0Day, pos1Day, x0data, x1data)
        x1wight = x1data.close / x0data.close
        pxedata = getStockByDate(data, calcBAYMDate(lastDay, tj1['y'], tj1['m'], True), code) * x1wight
        pdelta = abs((pxedata.close - xedata.close ) / xedata.close )
        if pdelta > delta:
            if len(gannList) <= 1:
                print('GannSubCheck None')
                return None
            else:
                print('GannSubCheck1', gannList, pos0Day, pos1Day, pdelta)
                return GannSubCheck(data, code, lastDay, posB1Day, gannList[1:], delta)
        else:
            print('GannSubCheck2', gannList, posB1Day, pdelta)
            rule = "%s:%s:Y%02d:M%02d" % (str(pos0Day)[:10], str(pos1Day)[:10], tj1['y'], tj1['m'])
            return {'pos0':pos0Day, 'pos1':pos1Day, 'w':x1wight, 'tj1':tj1, 'rule':rule}
    else:
        if base2 < firstDay:
            return None
        print('GannSubCheck3', base2)
        return GannSubCheck(data, code, lastDay, base2, gannList, delta)


In [ ]:
def forcast(code, lastDay, beforeStep, afterStep, delta = 0.1, predate = '1993-01-01', plot = None):
    if predate < str(data.index[0][0])[:10]:
        return None
    print('forcast', predate, str(data.index[0][0])[:10])
    col_name = predate.replace('-','')
    year = int(predate[:4])
    month = int(predate[6:7])
    day = int(predate[9:10])
    # outs=GannCheck(data, pd.Timestamp(1992,5,26), gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
    gannList = [{'y':15,'m':0},{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}]
    # gannList = [{'y':15,'m':0},{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':3},{'y':0,'m':1}]
    # gannList = [{'y':15,'m':0},{'y':7,'m':6},{'y':2,'m':6},{'y':1,'m':3},{'y':0,'m':7},{'y':0,'m':3},{'y':0,'m':1}]
    # gannList = [{'y':28,'m':0},{'y':15,'m':0},{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}]
    outs=GannCheck(data, code, lastDay, pd.Timestamp(year,month,day), delta = delta, gannBase = {'y':30,'m':0}, gannList = gannList)
    if outs == None:
        return None
    # outs
    # print('计算规则', outs['rule'])
    data[col_name] = 0.0
    x1wight=outs['w']
    # print(x1wight)
    gannBase=outs['tj1']
    # print(gannBase)
    # lastDay = data.index[-1][0]
    # lastDay
    # print(lastDay)
    # code='000001'
    firstDay = None
    for rday in range(-beforeStep, afterStep):
        ycday = lastDay + datetime.timedelta(rday)
        if firstDay == None:
            firstDay = ycday
        # print(ycday)
        preN = getStockByDate(data, calcBAYMDate(ycday, gannBase['y'], gannBase['m'], True), code) * x1wight
        preD = afterDate(preN.name[0], year=0, month = 7)
        try:
            rclose = data.at[(ycday,code),'close']
            # print(ycday)
            rclose = getStockByDate(data, ycday, code).close
            diff = preN.close - rclose
        except Exception as e:
    #         print(preN.name[0])
            rclose = 0.0
            diff = 0
    #     print(str(preD)[:10], int(preN.close), rclose, int(diff))
        if rclose == 0 and ycday < lastDay:
    #         print(ycday)
            continue
        else:
            if ycday.weekday() > 4:
                continue
            rclose = getStockByDate(data, ycday, code).close
    #     print(ycday.weekday())
        data.at[(ycday,code),col_name]  = preN.close
        data.at[(ycday,code),'close']  = rclose
        # data.at[]
    # data.tail(30)
    xlabel = []
    for x in data.loc[(firstDay,code):].index:
        xlabel.append(str(x[0])[5:10])
    # data.iloc[-24:].close.plot()
    # data.iloc[-24:][col_name].plot()
    # plt.show()
    if plot == None:
        return None

    plot.plot(xlabel, data.loc[(firstDay,code):][col_name], label="%s=>%s" % (predate, outs['rule']))
    return (firstDay, xlabel)


In [ ]:
code='000001'
idx='1'
beforeStep = 20
afterStep = 20
predict=['1992-05-26', '1997-05-12', '1996-12-11', '1996-01-19']
# predict=['1997-05-12', '1996-12-11', '1996-01-19']
# print(predict)

# argv = sys.argv[1:]
# code=argv[0]
# idx=argv[1]
# beforeStep = int(argv[2])
# afterStep = int(argv[3])
delta = 0.1
# predict=argv[5:]
print(delta, predict)

code='600581'
idx='0'

m=MongoIo()
if idx == '1':
    data=m.get_index_day(code, st_start = '1990-01-01')
else:
    data=m.get_stock_day(code, st_start = '1990-01-01')
lastDay = data.index[-1][0]
print(code, lastDay)
# print(data.head())
# data=m.get_index_day(code, st_start = '1990-01-01')
fig = plt.figure(figsize=(960/72,360/72))
# fig, ax = plt.subplots()
stockPlot = fig.add_subplot(1,1,1)
text = stockPlot.text(0.5, 0.5, 'event', ha='center', va='center', fontdict={'size': 20})

# text0 = plt.text(len_y-1,y[-1],str(y[-1]),fontsize = 10)
firstDay = None
xlabel = None
for predata in predict:
    print('step', predata)
    temp = forcast(code, lastDay, beforeStep, afterStep, predate = predata, plot = stockPlot, delta = delta)
    if firstDay == None and temp != None:
        (firstDay, xlabel) = temp
# forcast(code, col_name = '19970512', predate = '1997-05-12')

if firstDay != None:
    stockPlot.plot(xlabel, data.loc[(firstDay,code):]['close'], label="%s:REAL" % code, linewidth = '2', color = '#00FF00')

for label in stockPlot.xaxis.get_ticklabels():
    label.set_rotation(45)
# print(data.tail(30))
# fig.canvas.mpl_connect('scroll_event', scroll)
# fig.canvas.mpl_connect('motion_notify_event', motion)
# fig.canvas.mpl_connect('button_press_event', call_back)

plt.legend()
plt.show()

In [ ]:
data.head()

In [ ]:
data.filter(level)

In [ ]:
data2=data.reset_index()
data2['year'] = data2.date.dt.year
data2.iloc[data2[data2['year'] == 2002]['close'].argmax()]

# data2[data2['close']==data2['close'].groupby(data2.date.dt.year).max()]
# data2['close'].groupby(data2.date.dt.year)

# data2.close[data2['close'] == data2['close'].groupby(data2.date.dt.year).max()]
# data2.tail()

In [ ]:
data2.iloc[data2[data2['year'] == 2003]['close'].argmax()]

In [ ]:
data2.iloc[63]

In [ ]:
data.head()

In [ ]:
data['close'].groupby(level = 'date').idxmax()

In [ ]:
df2.iloc[data2.close.max()]

In [231]:
def h_l_line(p_df, t=21,period=10000,fn=None):
    """
    根据给定的周期找出最高最低点的日期和数据，然后计算对应的斐波纳契数据
    :param fn: 高低线输出到文件,如果文件参数为None则不输出到文件
    :param p_df:股票交易数据
    :param t:数据周期
    :param period:数据长度
    :return:有效数据点，包括股票代码，日期，高低点周期交易天数、高低点周期自然天数
    """
    if p_df is None or len(p_df)<t:
        return None
    # 获取最新的period条数据
    # df1 = p_df.tail(period).reset_index(drop=True)
    df1 = p_df[['close','high','low','trade_date','ts_code']].copy()
    df1['cv'] = 0 #添加一列为后续保留值准备
    high = df1['high']
    low = df1['low']

    # 保留数据的df
    data = pd.DataFrame([])
    #获取首行为有效的数据点,加入到保留数据框中
    df1.loc[0,'cv'] = df1.iloc[0].high #最高价作为当前价
    first = df1.iloc[0:1]
    data = data.append(first)

    #取第一个日期的最高值作为当前值,开始为0，默认为上涨周期
    ci=0
    cv=df1.iloc[ci].high
    cup=True

    #循环处理每一个周期
    n=0
    lt = t
    while ci<df1.index.max():
        n=n+1
        # 取含当前日期的一个周期的最高和最低价以及索引值,如果出现下一个周期中当前点成为了这个周期的最高和最低点即当前点未变化则
        # 在前周期长度上扩展1个周期,一旦出现拐点则恢复周期。
        # 周期超高了数据长度则结束，当前点加入到数据有效点中。
        # 为什么不是从下一个点找周期，因为下一个点开始的周期则一定存在一个高低点，而这个高低点和当前点的高点或低点比较后一定会
        # 出现一个拐点，有时候不一定有拐点存在,所以要包含当前点
        ih = high[ci:ci+lt].idxmax()
        il = low[ci:ci+lt].idxmin()
        ihv = df1.iloc[ih].high
        ilv = df1.iloc[il].low
        if (ih==ci) & (il==ci):
            #数据结束了吗?如果结束了则直接添加当前数据到数据点和最后一个数据到数据点
            if (ci+lt)>df1.index.max():
                # 数据结束了,最后一个数据是否要添加到数据点中，由循环结束时处理
                break
            else:
                # 三点重叠但数据为结束 , 周期延长重新计算
                lt = lt + t
                continue
        if cup:
            # 上涨阶段
            if (ihv >= cv) & (ci != ih):
                # 如果上升周期中最高价有更新则仍然上涨持续，上涨价格有效，下跌的价格为噪声
                ci = ih
                cv = ihv
                cup = True
            else:
                # 未持续上涨，则下跌价格有效，出现了转折，此时上一个价格成为转折点价格,恢复计算周期
                df1.loc[ci,'cv'] = cv
                data = data.append(df1.iloc[ci:ci + 1])
                ci = il
                cv = ilv
                cup = False
                lt = t
        else:
            # 下跌阶段
            if (ilv<=cv) & (ci != il):
                # 下跌阶段持续创新低，则下跌价格有效，上涨价格为噪声
                ci = il
                cv = ilv
                cup = False
            else:
                # 未持续下跌，此时转为上涨，上涨价格有效，此时上一个价格成为转折点价格,恢复计算周期
                df1.loc[ci, 'cv'] = cv
                data = data.append(df1.iloc[ci:ci + 1])
                ci = ih
                cv = ihv
                cup = True
                lt = t

        # print(df1.iloc[ci:ci+1])
        # print(n,ci,cv,cup,ih,il)

        # if last+t>=df1.index.max():
        #     # 最后计算恰好为最后一个周期，则直接加入最后一个周期进入数据有效点，并且结束循环
        #     last = df1.index.max()
        #     df1.loc[last, 'cv'] = df1.iloc[last].close
        #     data = data.append(df1.iloc[last:last + 1])
        #     break
    #结束了，把当前点加入到数据有效点中
    df1.loc[ci, 'cv'] = cv
    data = data.append(df1.iloc[ci:ci + 1])
    if ci != df1.index.max():
        # 当前点不是最后一个点，则把最后一个点加入到数据点中
        df1.loc[df1.index.max(), 'cv'] = df1.iloc[df1.index.max()].close
        data = data.append(df1.tail(1))

    data = data.reset_index(drop=False)
    # 计算高低点转换的交易日数量即时间周期
    data['period'] = (data['index'] - data['index'].shift(1)).fillna(0)
    # 计算日期的差值,将字符串更改为日期
    trade_date = pd.to_datetime(data['trade_date'],format='%Y-%m-%d')
    days = trade_date - trade_date.shift(1)
    # 填充后转换为实际的天数数字
    days = (days.fillna(pd.Timedelta(0))).apply(lambda x:x.days)
    data['days'] = days
    # 对日期进行转换
    data['trade_date']=trade_date.apply(lambda x:x.strftime('%Y-%m-%d'))
    return data


In [233]:
data=m.get_stock_day('000859', st_start = '1990-01-01')
data3=data.copy()
data3['trade_date'] = data3.index.levels[0]
data3['ts_code'] = data3.index.levels[1][0]
data3=data3.reset_index()
hldata=h_l_line(data3, t = 240)
predict = []
for x in hldata.index:
    predict.append(hldata.iloc[x].trade_date)
predict[0]

'1998-11-19'

In [235]:
hldata

,index,close,high,low,trade_date,ts_code,cv,period,days
0,0,12.08,12.50,11.03,1998-11-19,000859,12.50,0.0,0
1,318,15.64,15.99,14.50,2000-03-29,000859,15.99,318.0,496
2,346,10.23,10.61,10.15,2000-05-15,000859,10.15,28.0,47
3,605,13.34,13.52,13.28,2001-06-13,000859,13.52,259.0,394
4,755,7.66,7.66,6.54,2002-01-23,000859,6.54,150.0,224
5,1042,9.67,10.38,9.45,2003-04-16,000859,10.38,287.0,448
6,1702,1.81,1.83,1.78,2006-03-08,000859,1.78,660.0,1057
7,1992,11.74,12.00,10.90,2007-05-28,000859,12.00,290.0,446
8,2342,2.20,2.30,2.15,2008-11-04,000859,2.15,350.0,526
9,2597,9.50,10.33,9.38,2009-11-26,000859,10.33,255.0,387
